In [1]:
from sqlalchemy import create_engine

engine = create_engine(r"sqlite:///C:/Users/blinklet/Documents/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite")

In [2]:
from sqlalchemy import inspect

inspector = inspect(engine)

print(inspector.get_table_names())

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [3]:
from pprint import pprint

# Columns in table "Album"
pprint(inspector.get_columns("Album"))
print()

# Primary Key for table "Album"
pprint(inspector.get_pk_constraint("Album"))

[{'autoincrement': 'auto',
  'default': None,
  'name': 'AlbumId',
  'nullable': False,
  'primary_key': 1,
  'type': INTEGER()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'Title',
  'nullable': False,
  'primary_key': 0,
  'type': NVARCHAR(length=160)},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'ArtistId',
  'nullable': False,
  'primary_key': 0,
  'type': INTEGER()}]

{'constrained_columns': ['AlbumId'], 'name': None}


In [4]:
for table_name in inspector.get_table_names():

    print(f"Table = {table_name}")
    
    print(f"Columns = ", end="")
    col_names_list = []
    for col in inspector.get_columns(table_name):
        col_names_list.append(col['name'])
    print(*col_names_list, sep=", ")
    
    print(f"Primary Keys = ", end="")
    pk_list = inspector.get_pk_constraint(table_name)
    pk_name_list = pk_list["constrained_columns"]
    print(*pk_name_list, sep=", ")

    fk_list = inspector.get_foreign_keys(table_name)
    if fk_list:
        print(f"Foreign Keys:")
        fk_name_list = []
        fk_reftbl_list = []
        fk_refcol_list = []
        
        for fk in fk_list:
            fk_name_list.append(*fk['constrained_columns'])
            fk_reftbl_list.append(fk['referred_table'])
            fk_refcol_list.append(*fk['referred_columns'])
            
        fk_info = zip(fk_name_list, fk_reftbl_list, fk_refcol_list)
        
        for n, t, c in fk_info:
            print(f"    {n} ---> {t}:{c}")

    print()

Table = Album
Columns = Title, ArtistId, AlbumId
Primary Keys = AlbumId
Foreign Keys:
    ArtistId ---> Artist:ArtistId

Table = Artist
Columns = ArtistId, Name
Primary Keys = ArtistId

Table = Customer
Columns = CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
Primary Keys = CustomerId
Foreign Keys:
    SupportRepId ---> Employee:EmployeeId

Table = Employee
Columns = EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
Primary Keys = EmployeeId
Foreign Keys:
    ReportsTo ---> Employee:EmployeeId

Table = Genre
Columns = GenreId, Name
Primary Keys = GenreId

Table = Invoice
Columns = InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
Primary Keys = InvoiceId
Foreign Keys:
    CustomerId ---> Customer:CustomerId

Table = InvoiceLine
Columns = InvoiceLineId, InvoiceId, Tra

In [5]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

Base = automap_base()
Base.prepare(autoload_with=engine)

print(*Base.metadata.tables, sep=", ")

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, Track, MediaType, Playlist, PlaylistTrack


In [6]:
print(*Base.classes.keys(), sep=", ")

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, Track, MediaType, Playlist


In [7]:
def table_info(table_name):
    table = Base.metadata.tables[table_name]
    print(f"Table: {table_name}")
    print(f"Columns: ")
    for col in table.columns:
        print(f"   {col.name:15}", end="")
        if col.primary_key:
            print(f"*Primary Key*   ", end="")
        if col.foreign_keys:
            print(f"{col.foreign_keys}", end="")
        print()
    print()

In [8]:
table_info("PlaylistTrack")
table_info("Track")

Table: PlaylistTrack
Columns: 
   PlaylistId     *Primary Key*   {ForeignKey('Playlist.PlaylistId')}
   TrackId        *Primary Key*   {ForeignKey('Track.TrackId')}

Table: Track
Columns: 
   TrackId        *Primary Key*   
   Name           
   AlbumId        {ForeignKey('Album.AlbumId')}
   MediaTypeId    {ForeignKey('MediaType.MediaTypeId')}
   GenreId        {ForeignKey('Genre.GenreId')}
   Composer       
   Milliseconds   
   Bytes          
   UnitPrice      



In [9]:
Album = Base.classes.Album
Artist = Base.classes.Artist
Customer = Base.classes.Customer
Employee = Base.classes.Employee
Genre = Base.classes.Genre
Invoice = Base.classes.Invoice
InvoiceLine = Base.classes.InvoiceLine
MediaType = Base.classes.MediaType
Playlist = Base.classes.Playlist
Track = Base.classes.Track
playlisttrack = Base.metadata.tables['PlaylistTrack']

In [137]:
import pandas as pd

#session1 = Session(engine)

albums = pd.read_sql_table(table_name='Album', con=engine)

print(albums)

ValueError: Table Album.limit(1) not found

In [11]:
print(pd.options.display.max_rows)
print(pd.options.display.max_columns)

60
20


In [12]:
artists = pd.read_sql_table(table_name='Artist', con=engine)

print(artists.head())

   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains


In [13]:
df1 = pd.merge(left = albums, right = artists, how = 'inner')
print(df1.head())
print(df1.shape)

   AlbumId                                  Title  ArtistId       Name
0        1  For Those About To Rock We Salute You         1      AC/DC
1        4                      Let There Be Rock         1      AC/DC
2        2                      Balls to the Wall         2     Accept
3        3                      Restless and Wild         2     Accept
4        5                               Big Ones         3  Aerosmith
(347, 4)


In [14]:
df1.drop('ArtistId', axis=1).rename(columns = {'Name':'Artist'})

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,2,Balls to the Wall,Accept
3,3,Restless and Wild,Accept
4,5,Big Ones,Aerosmith
...,...,...,...
342,342,"Locatelli: Concertos for Violin, Strings and C...","Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
343,344,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,Nash Ensemble


In [15]:
df2 = (pd
     .merge(albums, artists)
     .drop('ArtistId', axis=1)
     .rename(columns = {'Name':'Artist'}))
print(df2.head())
print(df2.shape)

   AlbumId                                  Title     Artist
0        1  For Those About To Rock We Salute You      AC/DC
1        4                      Let There Be Rock      AC/DC
2        2                      Balls to the Wall     Accept
3        3                      Restless and Wild     Accept
4        5                               Big Ones  Aerosmith
(347, 3)


In [16]:
tracks = pd.read_sql_table(table_name='Track', con=engine)

print(tracks.head())
print(tracks.shape)

   TrackId                                     Name  AlbumId  MediaTypeId  \
0        1  For Those About To Rock (We Salute You)        1            1   
1        2                        Balls to the Wall        2            2   
2        3                          Fast As a Shark        3            2   
3        4                        Restless and Wild        3            2   
4        5                     Princess of the Dawn        3            2   

   GenreId                                           Composer  Milliseconds  \
0        1          Angus Young, Malcolm Young, Brian Johnson        343719   
1        1                                               None        342562   
2        1  F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...        230619   
3        1  F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...        252051   
4        1                         Deaffy & R.A. Smith-Diesel        375418   

      Bytes  UnitPrice  
0  11170334       0.99  
1   5510424 

In [17]:
df3 = (pd
     .merge(df2, tracks)
     .drop(['AlbumId','TrackId',
            'Bytes','UnitPrice',
            'MediaTypeId','GenreId'], axis=1)
     .rename(columns = {'Name':'Track', 
                        'Title':'Album',
                        'Milliseconds':'Length(ms)'}))

print(df3.shape)
df3.head().style.format(thousands=",")

(3503, 5)


,Album,Artist,Track,Composer,Length(ms)
0,For Those About To Rock We Salute You,AC/DC,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson","343,719"
1,For Those About To Rock We Salute You,AC/DC,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson","205,662"
2,For Those About To Rock We Salute You,AC/DC,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson","233,926"
3,For Those About To Rock We Salute You,AC/DC,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson","210,834"
4,For Those About To Rock We Salute You,AC/DC,Snowballed,"Angus Young, Malcolm Young, Brian Johnson","203,102"


In [18]:
print(f"Album table relationships")
print()
for relationship in inspect(Album).relationships:
    print(f"Relationship: {relationship}")
    print(f"Direction:    {relationship.direction}")
    print(f"Joined Table: {relationship.target}")
    print()

Album table relationships

Relationship: Album.artist
Direction:    symbol('MANYTOONE')
Joined Table: Artist

Relationship: Album.track_collection
Direction:    symbol('ONETOMANY')
Joined Table: Track



In [19]:
from sqlalchemy import select

session = Session(engine)

statement = select(Album)
first_album = session.scalar(statement)

print(f"Album:  {first_album.Title}")
print(f"Artist: {first_album.artist.Name}")
print(f"Tracks:")
for t in first_album.track_collection:
    print(f"    {t.Name}")

session.close()

Album:  For Those About To Rock We Salute You
Artist: AC/DC
Tracks:
    For Those About To Rock (We Salute You)
    Put The Finger On You
    Let's Get It Up
    Inject The Venom
    Snowballed
    Evil Walks
    C.O.D.
    Breaking The Rules
    Night Of The Long Knives
    Spellbound


C:\Users\blinklet\AppData\Local\Temp\ipykernel_23956\2387319706.py:11: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  for t in first_album.track_collection:


In [21]:
print(f"Get inner join of tables Album, Track, and Artist")
statement = (select(Album, Track, Artist)
     .join(Track)
     .join(Artist)
    )

df4 = pd.read_sql(statement, con=engine)

print(df4.shape)
display(df4.head())

Get inner join of tables Album, Track, and Artist
(3503, 14)


,AlbumId,Title,ArtistId,TrackId,Name,AlbumId_1,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,ArtistId_1,Name_1
0,1,For Those About To Rock We Salute You,1,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,1,AC/DC
1,2,Balls to the Wall,2,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,2,Accept
2,3,Restless and Wild,2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,2,Accept
3,3,Restless and Wild,2,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,2,Accept
4,3,Restless and Wild,2,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,2,Accept


In [35]:
print(f"Get inner join of tables Album, Track, and Artist")

statement = (select(Album.Title.label("Album"),
            Artist.Name.label("Artist"),
            Track.Name.label("Track"),
            Track.Composer, 
            Track.Milliseconds.label("Length"))
     .join(Track)
     .join(Artist)
    )

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
display(dataframe.head().style.format(thousands=","))

Get inner join of tables Album, Track, and Artist
(3503, 5)


,Album,Artist,Track,Composer,Length
0,For Those About To Rock We Salute You,AC/DC,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson","343,719"
1,Balls to the Wall,Accept,Balls to the Wall,None,"342,562"
2,Restless and Wild,Accept,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman","230,619"
3,Restless and Wild,Accept,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman","252,051"
4,Restless and Wild,Accept,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,"375,418"


In [24]:
albums = pd.read_sql_table(table_name='Album', con=engine)
artists = pd.read_sql_table(table_name='Artist', con=engine)
tracks = pd.read_sql_table(table_name='Track', con=engine)

df1 = (pd
     .merge(albums, artists)
     .rename(columns = {'Name':'Artist'}))

data = (pd
     .merge(df1, tracks)
     .rename(columns = {'Name':'Track', 
                        'Title':'Album',
                        'Milliseconds':'Length'}))

print(f"Longest track: {data.Length.max()}")
print(f"Shortest track: {data.Length.min()}")
print(f"How many blanks in Composer column?: "
      f"{data.Composer.isnull().sum()}")
print(f"Track length mean: {data.Length.mean()}")
print(f"Track length median: {data.Length.median()}")
print(f"Artist mode: {data.Artist.mode()[0]}")
print(f"Correlation between Length and UnitPrice: "
      f"{data['Length'].corr(data['UnitPrice'])}")
print(f"Track length standard deviation: "
      f"{data.Length.std()}")

Longest track: 5286953
Shortest track: 1071
How many blanks in Composer column?: 978
Track length mean: 393599.2121039109
Track length median: 255634.0
Artist mode: Iron Maiden
Correlation between Length and UnitPrice: 0.9317964749112725
Track length standard deviation: 535005.4352066235


In [25]:
from tabulate import tabulate

print(f"Track length standard deviation for a sample of artists:")
print(tabulate(
        data.groupby(['Artist'])['Length']
        .std()
        .dropna()
        .sample(3)
        .to_frame(), 
        headers = ['Artist','Track Length\nStd Dev'], 
        tablefmt="grid"))
print()
print(f"Longest tracks, with artist name:")
print(tabulate(
        data[['Track','Length','Artist']]
        .nlargest(3, 'Length'), 
        headers="keys", 
        tablefmt='grid', 
        showindex=False))
print()
print(f"Number of tracks per artist, from a sample of artists::")
print(tabulate(
        data
        .groupby('Artist')['Track']
        .count()
        .sample(3)
        .to_frame(), 
        headers = ['Artist','# Tracks'], 
        tablefmt="grid"))
print()
print(f"Shortest track by Artist=Guns N' Roses: ")
gnr = data.loc[data['Artist'] == "Guns N' Roses"]
gnr_shortest = gnr[['Track','Length']].nsmallest(3, 'Length')
print(tabulate(
        gnr_shortest, 
        headers="keys", 
        tablefmt='grid', 
        showindex=False))       

Track length standard deviation for a sample of artists:
+------------------------------------------------+----------------+
| Artist                                         |   Track Length |
|                                                |        Std Dev |
+================================================+================+
| Deep Purple                                    |         170969 |
+------------------------------------------------+----------------+
| Michael Tilson Thomas & San Francisco Symphony |         202906 |
+------------------------------------------------+----------------+
| Metallica                                      |         105800 |
+------------------------------------------------+----------------+

Longest tracks, with artist name:
+-----------------------------+----------+--------------------------------+
| Track                       |   Length | Artist                         |
+=============================+==========+================================+


In [26]:
from sqlalchemy import select, func

with Session(engine) as session:
    statement = select(func.max(Track.Milliseconds))
    length = session.scalar(statement)

print(f"Longest track length: {length}")



Longest track length: 5286953


In [43]:
with Session(engine) as session:
    result = session.execute(statement)
    print(result)
    print(type(result))

<class 'sqlalchemy.engine.result.ChunkedIteratorResult'>


In [45]:
with Session(engine) as session:
    result = session.execute(statement)
    print(next(result))
    print(next(result))
    print(type(result))

('For Those About To Rock We Salute You', 'AC/DC', 'For Those About To Rock (We Salute You)', 'Angus Young, Malcolm Young, Brian Johnson', 343719)
('Balls to the Wall', 'Accept', 'Balls to the Wall', None, 342562)
<class 'sqlalchemy.engine.result.ChunkedIteratorResult'>


In [49]:
with Session(engine) as session:
    result = session.execute(statement)
    x = next(result).Artist
    print(x)
    print(type(x))

AC/DC
<class 'str'>


In [56]:
with Session(engine) as session:
    limit = 4
    result = session.execute(statement)
    for index, item in enumerate(result, start=1):
        print(item)
        if index == limit:
            break

('For Those About To Rock We Salute You', 'AC/DC', 'For Those About To Rock (We Salute You)', 'Angus Young, Malcolm Young, Brian Johnson', 343719)
('Balls to the Wall', 'Accept', 'Balls to the Wall', None, 342562)
('Restless and Wild', 'Accept', 'Fast As a Shark', 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 230619)
('Restless and Wild', 'Accept', 'Restless and Wild', 'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman', 252051)


In [72]:
table = []
limit = 4

with Session(engine) as session:
    result = session.execute(statement)
    for index, item in enumerate(result, start=1):
        table.append(item)
        if index == limit:
            break
    headers = session.execute(statement).keys()
    print(type(headers))
    
print(type(headers))
print(table)

<class 'sqlalchemy.engine.result.RMKeyView'>
<class 'sqlalchemy.engine.result.RMKeyView'>
[('For Those About To Rock We Salute You', 'AC/DC', 'For Those About To Rock (We Salute You)', 'Angus Young, Malcolm Young, Brian Johnson', 343719), ('Balls to the Wall', 'Accept', 'Balls to the Wall', None, 342562), ('Restless and Wild', 'Accept', 'Fast As a Shark', 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 230619), ('Restless and Wild', 'Accept', 'Restless and Wild', 'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman', 252051)]


In [82]:
from collections import namedtuple

with Session(engine) as session:
    headers = session.execute(statement).keys()
    table = session.execute(statement).fetchmany(4)

print(type(headers))
print(next(iter(headers)))
print(type(table))
for row in table:
    print(type(row))

<class 'sqlalchemy.engine.result.RMKeyView'>
<class '__main__.tuple'>
Album
<class 'list'>
<class 'sqlalchemy.engine.row.Row'>
<class 'sqlalchemy.engine.row.Row'>
<class 'sqlalchemy.engine.row.Row'>
<class 'sqlalchemy.engine.row.Row'>


In [97]:
from tabulate import tabulate

with Session(engine) as session:
    print(session.execute(statement))
    print()
    print(session.execute(statement).first())
    print()
    print(session.execute(statement).fetchmany(2))
    print()
    print(session.execute(statement.limit(2)).fetchall())
    print()
    print(session.scalars(statement))
    print()
    print(session.scalars(statement).first())
    print()
    print(session.scalars(statement).fetchmany(2))
    print()
    print(session.scalars(statement.limit(2)).fetchall())
    print()
    print(session.scalar(statement))


('For Those About To Rock We Salute You', 'AC/DC', 'For Those About To Rock (We Salute You)', 'Angus Young, Malcolm Young, Brian Johnson', 343719)

[('For Those About To Rock We Salute You', 'AC/DC', 'For Those About To Rock (We Salute You)', 'Angus Young, Malcolm Young, Brian Johnson', 343719), ('Balls to the Wall', 'Accept', 'Balls to the Wall', None, 342562)]

[('For Those About To Rock We Salute You', 'AC/DC', 'For Those About To Rock (We Salute You)', 'Angus Young, Malcolm Young, Brian Johnson', 343719), ('Balls to the Wall', 'Accept', 'Balls to the Wall', None, 342562)]


For Those About To Rock We Salute You

['For Those About To Rock We Salute You', 'Balls to the Wall']

['For Those About To Rock We Salute You', 'Balls to the Wall']

For Those About To Rock We Salute You


In [124]:
print(f"Experiment with returned data types\n")

with Session(engine) as session:
    query = session.execute(statement.filter(Artist.Name == 'Alice In Chains'))
    headings = query.keys()
    result = query.fetchmany(4)
    
    query = session.execute(statement.filter(Artist.Name == 'Alice In Chains').limit(4))
    result_as_dicts = query.mappings().all()

print(result)

print()
for row in result:
    print(f"Track name: {row.Track}\tComposer: {row.Composer}")

print()
for row in result_as_dicts:
    print(row)

print()
for row in result:
    print(f"Track name: {row['Track']}\tComposer: {row['Composer']}")
    


Experiment with returned data types

[('Facelift', 'Alice In Chains', 'We Die Young', 'Jerry Cantrell', 152084), ('Facelift', 'Alice In Chains', 'Man In The Box', 'Jerry Cantrell, Layne Staley', 286641), ('Facelift', 'Alice In Chains', 'Sea Of Sorrow', 'Jerry Cantrell', 349831), ('Facelift', 'Alice In Chains', 'Bleed The Freak', 'Jerry Cantrell', 241946)]

Track name: We Die Young	Composer: Jerry Cantrell
Track name: Man In The Box	Composer: Jerry Cantrell, Layne Staley
Track name: Sea Of Sorrow	Composer: Jerry Cantrell
Track name: Bleed The Freak	Composer: Jerry Cantrell

{'Album': 'Facelift', 'Artist': 'Alice In Chains', 'Track': 'We Die Young', 'Composer': 'Jerry Cantrell', 'Length': 152084}
{'Album': 'Facelift', 'Artist': 'Alice In Chains', 'Track': 'Man In The Box', 'Composer': 'Jerry Cantrell, Layne Staley', 'Length': 286641}
{'Album': 'Facelift', 'Artist': 'Alice In Chains', 'Track': 'Sea Of Sorrow', 'Composer': 'Jerry Cantrell', 'Length': 349831}
{'Album': 'Facelift', 'Artist':

In [129]:
print(f"Experiment with returned data types\n")

with Session(engine) as session:
    query = session.execute(statement.filter(Artist.Name == 'Alice In Chains'))
    headings = query.keys()
    result = query.fetchmany(4)
    
print(result)

print()
for row in result:
    print(f"Track name: {row.Track}\tComposer: {row.Composer}")

Experiment with returned data types

[('Facelift', 'Alice In Chains', 'We Die Young', 'Jerry Cantrell', 152084), ('Facelift', 'Alice In Chains', 'Man In The Box', 'Jerry Cantrell, Layne Staley', 286641), ('Facelift', 'Alice In Chains', 'Sea Of Sorrow', 'Jerry Cantrell', 349831), ('Facelift', 'Alice In Chains', 'Bleed The Freak', 'Jerry Cantrell', 241946)]

Track name: We Die Young	Composer: Jerry Cantrell
Track name: Man In The Box	Composer: Jerry Cantrell, Layne Staley
Track name: Sea Of Sorrow	Composer: Jerry Cantrell
Track name: Bleed The Freak	Composer: Jerry Cantrell


In [130]:
with Session(engine) as session:
    query = session.execute(statement)
    result_headings = query.keys()
    result = query.fetchmany(5)

for x in result:
    print(x)

('For Those About To Rock We Salute You', 'AC/DC', 'For Those About To Rock (We Salute You)', 'Angus Young, Malcolm Young, Brian Johnson', 343719)
('Balls to the Wall', 'Accept', 'Balls to the Wall', None, 342562)
('Restless and Wild', 'Accept', 'Fast As a Shark', 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 230619)
('Restless and Wild', 'Accept', 'Restless and Wild', 'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman', 252051)
('Restless and Wild', 'Accept', 'Princess of the Dawn', 'Deaffy & R.A. Smith-Diesel', 375418)


In [ ]:
print("similar to pandas select() method")
print()
with Session(engine) as session:
    result = session.execute(statement.order_by(func.random()).limit(2)).fetchall()

print(result)

In [ ]:
length = (session1
            .query(func.min(Track.Milliseconds))
            .scalar()
         )

print(f"Shortest track length: {length}")

In [141]:
statement = (select(Artist))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
print(dataframe.head(5))

(275, 2)
   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains


In [142]:
statement = (select(Artist).filter(Artist.Name=='Alice In Chains'))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
print(dataframe.head(5))

(1, 2)
   ArtistId             Name
0         5  Alice In Chains


In [148]:
statement = (select(Artist).order_by(func.random()).limit(5))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
print(dataframe.head(5))

(5, 2)
   ArtistId                Name
0       247  The King's Singers
1       191         Nação Zumbi
2       251            Fretwork
3        36             O Rappa
4        94        Jimi Hendrix


In [176]:
from sqlalchemy import select

session = Session(engine)

statement = select(Album)
first_album = session.scalars(statement.limit(3))
print(dir(first_album))

for x in first_album:

    print(f"Album:  {x.Title}")
    print(f"Artist: {x.artist.Name}")
    print(f"Tracks:")
    for t in x.track_collection:
        print(f"    {t.Name}")

session.close()

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_allrows', '_assert_no_memoizations', '_attributes', '_column_slices', '_fetchall_impl', '_fetchiter_impl', '_fetchmany_impl', '_fetchone_impl', '_generate', '_generate_rows', '_is_cursor', '_iter_impl', '_iterator_getter', '_manyrow_getter', '_memoized_keys', '_metadata', '_next_impl', '_onerow_getter', '_only_one_row', '_post_creational_filter', '_raw_all_rows', '_real_result', '_reset_memoizations', '_row_getter', '_set_memoized_attribute', '_soft_close', '_soft_closed', '_unique_filter_state', '_unique_strategy', 'all', 'close', 'closed', 'fetchall', 'fetchmany', 'first', 'memoized_attribut

In [187]:
statement = (select(Album.Title.label("Album"),
            Artist.Name.label("Artist"),
            Track.Name.label("Track"),
            Track.Composer, 
            Track.Milliseconds.label("Length"))
     .join(Track)
     .join(Artist)
    )

session = Session(engine)

first_album = session.execute(statement.limit(3))

for x in first_album:
    print(f"Album:  {x.Album}\tArtist: {x.Artist}\tTrack: {x.Track}")

print()

session.close()

Album:  For Those About To Rock We Salute You	Artist: AC/DC	Track: For Those About To Rock (We Salute You)
Album:  Balls to the Wall	Artist: Accept	Track: Balls to the Wall
Album:  Restless and Wild	Artist: Accept	Track: Fast As a Shark



In [225]:
print(f"Names of all the tracks purchased by each customer")

statement = (select(Customer.FirstName,
                    Customer.LastName,
                    Customer.Country,
                    InvoiceLine.Quantity,
                    InvoiceLine.UnitPrice,
                    Track.Name,
                    Album.Title,
                    Artist.Name
                    )
                .join_from(Customer, Invoice)
                .join_from(Invoice, InvoiceLine)
                .join_from(InvoiceLine, Track)
                .join_from(Track, Album)
                .join_from(Album, Artist))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
display(dataframe.head(10).style.format(thousands=","))

Names of all the tracks purchased by each customer


InvalidRequestError: Don't know how to join to <Mapper at 0x173f1e5dad0; Customer>. Please use the .select_from() method to establish an explicit left side, as well as providing an explicit ON clause if not present already to help resolve the ambiguity.

In [234]:
print(f"Names of all the tracks purchased by each customer")

statement = (select(Customer.FirstName,
                    Customer.LastName,
                    Customer.Country,
                    Track.Name.label("Track"),
                    Album.Title.label("Album"),
                    Artist.Name.label("Artist"),
                    InvoiceLine.Quantity,
                    InvoiceLine.UnitPrice
                    )
                .join_from(InvoiceLine, Invoice)
                .join_from(Invoice, Customer)
                .join_from(InvoiceLine, Track)
                .join_from(Track, Album)
                .join_from(Album, Artist))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
display(dataframe.head(5).style.format(thousands=","))

Names of all the tracks purchased by each customer
(2240, 8)


,FirstName,LastName,Country,Track,Album,Artist,Quantity,UnitPrice
0,Leonie,Köhler,Germany,Balls to the Wall,Balls to the Wall,Accept,1,0.990000
1,Leonie,Köhler,Germany,Restless and Wild,Restless and Wild,Accept,1,0.990000
2,Bjørn,Hansen,Norway,Put The Finger On You,For Those About To Rock We Salute You,AC/DC,1,0.990000
3,Bjørn,Hansen,Norway,Inject The Venom,For Those About To Rock We Salute You,AC/DC,1,0.990000
4,Bjørn,Hansen,Norway,Evil Walks,For Those About To Rock We Salute You,AC/DC,1,0.990000


In [243]:
print(f"Names of all the tracks purchased by each customer")

statement = (select(Playlist.Name.label("Playlist"),
                    Track.Name.label("Track"),
                    Album.Title.label("Album"),
                    Artist.Name.label("Artist")
                    )
                .join_from(Playlist, playlisttrack)
                .join_from(playlisttrack, Track)
                .join_from(Track, Album)
                .join_from(Album, Artist))

dataframe = pd.read_sql(sql=statement, con=engine)

print(dataframe.shape)
display(dataframe.head(5).style.format(thousands=","))

Names of all the tracks purchased by each customer
(8715, 4)


,Playlist,Track,Album,Artist
0,Music,"Band Members Discuss Tracks from ""Revelations""",Revelations,Audioslave
1,Music,Revelations,Revelations,Audioslave
2,Music,One and the Same,Revelations,Audioslave
3,Music,Sound of a Gun,Revelations,Audioslave
4,Music,Until We Fall,Revelations,Audioslave


In [257]:
statement = (select(Playlist.Name.label("Playlist"),
                    Track.Name.label("Track"),
                    Album.Title.label("Album"),
                    Artist.Name.label("Artist")
                    )
                .join_from(Playlist, playlisttrack)
                .join_from(playlisttrack, Track)
                .join_from(Track, Album)
                .join_from(Album, Artist))

dataframe = pd.read_sql(sql=statement.order_by(func.random()).limit(5), con=engine)

print(dataframe.shape)
display(dataframe.head(5))

(5, 4)


,Playlist,Track,Album,Artist
0,90’s Music,Chaos-Control,[1997] Black Light Syndrome,"Terry Bozzio, Tony Levin & Steve Stevens"
1,Music,Realword,Chemical Wedding,Bruce Dickinson
2,Classical,"Symphony No. 2, Op. 16 - ""The Four Temperamen...",Nielsen: The Six Symphonies,Göteborgs Symfoniker & Neeme Järvi
3,90’s Music,Rats,Vs.,Pearl Jam
4,Music,"A Midsummer Night's Dream, Op.61 Incidental Mu...",Mendelssohn: A Midsummer Night's Dream,Philharmonia Orchestra & Sir Neville Marriner


In [248]:
from sqlalchemy import create_engine, inspect, select
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import pandas as pd
from tabulate import tabulate

engine = create_engine(r"sqlite:///C:/Users/blinklet/Documents/chinook-database/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite")
Base = automap_base()
Base.prepare(autoload_with=engine)

Album = Base.classes.Album
Artist = Base.classes.Artist
Customer = Base.classes.Customer
Employee = Base.classes.Employee
Genre = Base.classes.Genre
Invoice = Base.classes.Invoice
InvoiceLine = Base.classes.InvoiceLine
MediaType = Base.classes.MediaType
Playlist = Base.classes.Playlist
Track = Base.classes.Track
playlisttrack = Base.metadata.tables['PlaylistTrack']

statement = (select(Playlist.Name.label("Playlist"),
                Track.Name.label("Track"),
                Album.Title.label("Album"),
                Artist.Name.label("Artist")
                )
            .join_from(Playlist, playlisttrack)
            .join_from(playlisttrack, Track)
            .join_from(Track, Album)
            .join_from(Album, Artist))

with Session(engine) as session4:
    headers = session.execute(statement).keys()
    table = session.execute(statement).fetchmany(4)

print(tabulate(table, headers, tablefmt='grid'))

+------------+------------------------------------------------+-------------+------------+
| Playlist   | Track                                          | Album       | Artist     |
+============+================================================+=============+============+
| Music      | Band Members Discuss Tracks from "Revelations" | Revelations | Audioslave |
+------------+------------------------------------------------+-------------+------------+
| Music      | Revelations                                    | Revelations | Audioslave |
+------------+------------------------------------------------+-------------+------------+
| Music      | One and the Same                               | Revelations | Audioslave |
+------------+------------------------------------------------+-------------+------------+
| Music      | Sound of a Gun                                 | Revelations | Audioslave |
+------------+------------------------------------------------+-------------+------------+